**Karan Kumar Sethi**
**Roll No:- 22CS30034**


**Importing Necessary modules**

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold


**Neural Network Class Implementation**

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        # Initialize neural network parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Randomly initialize weights (He initialization)
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * np.sqrt(2.0 / self.input_size)
        self.b1 = np.zeros((1, self.hidden_size))  # Bias for hidden layer
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * np.sqrt(2.0 / self.hidden_size)
        self.b2 = np.zeros((1, self.output_size))  # Bias for output layer

    def sigmoid(self, z):
        """ Sigmoid activation function """
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        """ Derivative of sigmoid function (used in backpropagation) """
        return z * (1 - z)

    def forward(self, X):
        """ Forward propagation to calculate predicted output """
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)  # Hidden layer activations
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        output = self.sigmoid(self.z2)  # Output layer activations
        return output

    def backprop(self, X, y, output):
        """ Backpropagation to compute gradients and update weights """
        # Calculate error in output layer
        output_error = y - output  # Error in output
        output_delta = output_error * self.sigmoid_derivative(output)  # Gradient of loss with respect to output

        # Calculate error in hidden layer
        hidden_error = np.dot(output_delta, self.W2.T)  # Error propagated to hidden layer
        hidden_delta = hidden_error * self.sigmoid_derivative(self.a1)  # Gradient of hidden layer activations

        # Update weights and biases
        self.W2 += self.learning_rate * np.dot(self.a1.T, output_delta)  # Update W2
        self.b2 += self.learning_rate * np.sum(output_delta, axis=0, keepdims=True)  # Update bias for output layer
        self.W1 += self.learning_rate * np.dot(X.T, hidden_delta)  # Update W1
        self.b1 += self.learning_rate * np.sum(hidden_delta, axis=0, keepdims=True)  # Update bias for hidden layer

    def train(self, X, y, epochs):
        """ Train the model for a fixed number of epochs """
        for epoch in range(epochs):
            output = self.forward(X)
            self.backprop(X, y, output)

    def calculate_loss(self, y, output):
        """ Calculate the Mean Squared Error (MSE) loss """
        return np.mean((y - output) ** 2)


**Data Preprocessing and Normalization**

In [ ]:
def load_and_preprocess_data():
    """ Load and normalize the Boston Housing dataset """
    df = pd.read_csv('housing.csv')
    X_data = df.drop(columns=['MEDV']).to_numpy()
    y_data = df['MEDV'].to_numpy().reshape(-1, 1)

    # Normalize data using min-max scaling (optional, adjust as needed)
    X_data = (X_data - X_data.min(axis=0)) / (X_data.max(axis=0) - X_data.min(axis=0))
    y_data = (y_data - y_data.min()) / (y_data.max() - y_data.min())

    return X_data, y_data

**Cross-validation function**

In [ ]:
def perform_k_fold_cross_validation(X, y, hidden_size, learning_rate, epochs, k_folds=5):
    """ Perform k-fold cross-validation on the dataset """
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=1)
    losses = []

    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize and train neural network
        nn = NeuralNetwork(input_size=X.shape[1], hidden_size=hidden_size, output_size=1, learning_rate=learning_rate)
        nn.train(X_train, y_train, epochs)

        # Calculate loss on test set
        output = nn.forward(X_test)
        loss = nn.calculate_loss(y_test, output)
        losses.append(loss)

    return np.mean(losses)

**Main function to run experiments with user inputs**

In [ ]:
def run_experiments():
    """ Main experiment loop for different configurations of the neural network """
    # Load and preprocess dataset
    X_data, y_data = load_and_preprocess_data()

    # Configuration settings from user input
    hidden_size = int(input("Enter number of neurons in the hidden layer: "))
    learning_rate = float(input("Enter the learning rate: "))
    epochs = 1000

    print("5-fold cross-validation results:")
    avg_loss_5fold = perform_k_fold_cross_validation(X_data, y_data, hidden_size, learning_rate, epochs, k_folds=5)
    print(f'Configuration: {hidden_size} hidden neurons, learning rate: {learning_rate}, Average Loss (5-Fold CV): {avg_loss_5fold}')

    print("10-fold cross-validation results:")
    avg_loss_10fold = perform_k_fold_cross_validation(X_data, y_data, hidden_size, learning_rate, epochs, k_folds=10)
    print(f'Configuration: {hidden_size} hidden neurons, learning rate: {learning_rate}, Average Loss (10-Fold CV): {avg_loss_10fold}')

**Run the experiments**

In [ ]:
runExpr = input("Do you want to run the Experiment (y/n): ")
while(runExpr.lower() == 'y'):
    run_experiments()
    runExpr = input("Do you want to run Experiment again (y/n): ")
